![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
![table example](img/columns.png)

### Terminology

- Schema
- Primary Key
- Foreign Key
- Structured queries
- Views

***
### SQLite

![sqlite](img/SQLite-Python.jpg)

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to an Oracle database server there is a different package, `cx_Oracle`. The documentation for that package is [here](https://oracle.github.io/python-cx_Oracle/)

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [32]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [33]:
cursor.description

(('name', None, None, None, None, None, None),)

In [6]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [8]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [39]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [29]:
cursor.fetchone()

(9,
 '10',
 'Thule Air Base',
 'Thule',
 'Greenland',
 'THU',
 'BGTL',
 '76.531203',
 '-68.703161',
 '251',
 '-4',
 'E',
 'America/Thule')

In [30]:
cursor.fetchmany(4)

[(10,
  '11',
  'Akureyri',
  'Akureyri',
  'Iceland',
  'AEY',
  'BIAR',
  '65.659994',
  '-18.072703',
  '6',
  '0',
  'N',
  'Atlantic/Reykjavik'),
 (11,
  '12',
  'Egilsstadir',
  'Egilsstadir',
  'Iceland',
  'EGS',
  'BIEG',
  '65.283333',
  '-14.401389',
  '76',
  '0',
  'N',
  'Atlantic/Reykjavik'),
 (12,
  '13',
  'Hornafjordur',
  'Hofn',
  'Iceland',
  'HFN',
  'BIHN',
  '64.295556',
  '-15.227222',
  '24',
  '0',
  'N',
  'Atlantic/Reykjavik'),
 (13,
  '14',
  'Husavik',
  'Husavik',
  'Iceland',
  'HZK',
  'BIHU',
  '65.952328',
  '-17.425978',
  '48',
  '0',
  'N',
  'Atlantic/Reykjavik')]

#### Use description

In [31]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [34]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [50]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [51]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

In [38]:
cursor.execute("""select "Flatiron School\"""").fetchall()

[('Flatiron School',)]

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [55]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [61]:
cursor.execute("""
SELECT *
FROM airports
""")
print(*cursor.description, sep='\n')

('index', None, None, None, None, None, None)
('id', None, None, None, None, None, None)
('name', None, None, None, None, None, None)
('city', None, None, None, None, None, None)
('country', None, None, None, None, None, None)
('code', None, None, None, None, None, None)
('icao', None, None, None, None, None, None)
('latitude', None, None, None, None, None, None)
('longitude', None, None, None, None, None, None)
('altitude', None, None, None, None, None, None)
('offset', None, None, None, None, None, None)
('dst', None, None, None, None, None, None)
('timezone', None, None, None, None, None, None)


### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [65]:
# Which countries have the highest amount of active airports?
cursor.execute("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'Y'
GROUP BY country
ORDER BY 2 DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)
('Australia', 26)
('China', 25)
('Spain', 24)
('Brazil', 23)
('France', 22)


In [67]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'N'
GROUP BY country
ORDER BY 2 DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)
('Spain', 142)
('France', 97)
('Germany', 94)
('South Africa', 81)
('Nigeria', 80)


In [70]:
# What about airports by timezones?
cursor.execute("""
SELECT timezone, count(*) AS count
FROM airports
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)
('America/Los_Angeles', 226)
('Asia/Chongqing', 222)
('Europe/London', 193)
('America/Toronto', 159)
('Asia/Calcutta', 141)


In [78]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [71]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [72]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()
#vs
shelter_data[shelter_data['Name']=="Max"]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
83,A797258,Max,06/19/2019 12:58:00 PM,06/19/2019 12:58:00 PM,02/11/2018,Transfer,Partner,Dog,Intact Male,1 year,Rottweiler Mix,Black/Tan
154,A797814,Max,06/18/2019 12:52:00 PM,06/18/2019 12:52:00 PM,06/18/2017,Return to Owner,NaN,Dog,Intact Male,2 years,Pug,Tan/Black
198,A797341,Max,06/17/2019 05:12:00 PM,06/17/2019 05:12:00 PM,03/12/2019,Adoption,NaN,Cat,Neutered Male,3 months,Domestic Shorthair,Cream Tabby/White
415,A680607,Max,06/14/2019 11:50:00 AM,06/14/2019 11:50:00 AM,04/04/2014,Return to Owner,NaN,Dog,Neutered Male,5 years,Chihuahua Shorthair Mix,Tan/White
481,A797339,Max,06/13/2019 11:39:00 AM,06/13/2019 11:39:00 AM,06/12/2017,Return to Owner,NaN,Dog,Intact Male,2 years,Welsh Terrier/Miniature Poodle,Tan/Gray
732,A780200,Max,06/10/2019 10:53:00 AM,06/10/2019 10:53:00 AM,09/11/2014,Rto-Adopt,NaN,Dog,Neutered Male,4 years,German Shepherd Mix,Brown/Black
756,A793073,Max,06/09/2019 05:52:00 PM,06/09/2019 05:52:00 PM,06/02/2018,Adoption,NaN,Dog,Neutered Male,1 year,Boxer Mix,Brown/Tricolor
1106,A744193,Max,06/04/2019 03:30:00 PM,06/04/2019 03:30:00 PM,09/27/2016,Adoption,NaN,Dog,Neutered Male,2 years,Cairn Terrier Mix,Tan/White
1169,A794951,Max,06/03/2019 07:30:00 PM,06/03/2019 07:30:00 PM,05/14/2017,Rto-Adopt,NaN,Dog,Intact Male,2 years,Labrador Retriever,Red
1328,A796351,Max,06/01/2019 04:18:00 PM,06/01/2019 04:18:00 PM,05/31/2010,Return to Owner,NaN,Cat,Neutered Male,9 years,Domestic Shorthair,Black/White


In [74]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,animal_type,Sex upon Outcome,Age upon Outcome,Breed,Color
1,A535405,Charlie,06/20/2019 12:53:00 PM,06/20/2019 12:53:00 PM,08/23/2007,Adoption,NaN,Dog,Neutered Male,11 years,Labrador Retriever/Basset Hound,Black
2,A778902,Eleanor R.,06/20/2019 12:52:00 PM,06/20/2019 12:52:00 PM,08/22/2016,Adoption,NaN,Dog,Spayed Female,2 years,German Shepherd Mix,Tricolor
6,A720901,Axle,06/20/2019 12:05:00 PM,06/20/2019 12:05:00 PM,02/17/2013,Return to Owner,NaN,Dog,Intact Male,6 years,Staffordshire Mix,Black
12,A797960,Cooper,06/20/2019 11:53:00 AM,06/20/2019 11:53:00 AM,01/13/2016,Return to Owner,NaN,Dog,Intact Male,3 years,Golden Retriever,Red
13,A797985,Jewel,06/20/2019 11:51:00 AM,06/20/2019 11:51:00 AM,06/19/2017,Return to Owner,NaN,Dog,Intact Female,2 years,Pit Bull,Blue/White


## Goal 6: Transfering from sqlite to pandas

In [76]:
pd_con = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [77]:
# Which countries have the highest amount of active airports?
df = pd.read_sql_query("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'Y'
GROUP BY country
ORDER BY 2 DESC
LIMIT 10
""", pd_con)
df

,country,count
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34
5,Australia,26
6,China,25
7,Spain,24
8,Brazil,23
9,France,22


In [79]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [82]:
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [87]:
# How many tables are in the database?
cur.execute("SELECT count(*) as table_count FROM sqlite_master WHERE type = 'table';").fetchone()

(11,)

In [99]:
# What's the primary key of each table?
table_names = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()
for table_name in table_names:
    tname = table_name[0]
    # table_name is still a tuple of length 1, so we have to unpack to get the table name value
    results = cur.execute(f"PRAGMA table_info({tname});").fetchall()

    # column name is 2nd column,
    # primary key is the 4th column, according to https://www.sqlite.org/pragma.html#pragma_table_info
    pk_column_name = [x[1] for x in results if x[3] == 1]
    print(f"{tname}.{pk_column_name[0]}")

Album.AlbumId
Artist.ArtistId
Customer.CustomerId
Employee.EmployeeId
Genre.GenreId
Invoice.InvoiceId
InvoiceLine.InvoiceLineId
MediaType.MediaTypeId
Playlist.PlaylistId
PlaylistTrack.PlaylistId
Track.TrackId


In [104]:
# What foreign keys join the tables together?
table_names = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()
for table_name in table_names:
    tname = table_name[0]
    # table_name is still a tuple of length 1, so we have to unpack to get the table name value
    results = cur.execute(f"PRAGMA foreign_key_list({tname});").fetchall()
    if results:
        print(tname, 'table:')
        print(*results, sep='\n')

Album table:
(0, 0, 'Artist', 'ArtistId', 'ArtistId', 'NO ACTION', 'NO ACTION', 'NONE')
Customer table:
(0, 0, 'Employee', 'SupportRepId', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')
Employee table:
(0, 0, 'Employee', 'ReportsTo', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')
Invoice table:
(0, 0, 'Customer', 'CustomerId', 'CustomerId', 'NO ACTION', 'NO ACTION', 'NONE')
InvoiceLine table:
(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE')
(1, 0, 'Invoice', 'InvoiceId', 'InvoiceId', 'NO ACTION', 'NO ACTION', 'NONE')
PlaylistTrack table:
(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE')
(1, 0, 'Playlist', 'PlaylistId', 'PlaylistId', 'NO ACTION', 'NO ACTION', 'NONE')
Track table:
(0, 0, 'MediaType', 'MediaTypeId', 'MediaTypeId', 'NO ACTION', 'NO ACTION', 'NONE')
(1, 0, 'Genre', 'GenreId', 'GenreId', 'NO ACTION', 'NO ACTION', 'NONE')
(2, 0, 'Album', 'AlbumId', 'AlbumId', 'NO ACTION', 'NO ACTION', 'NONE')


In [109]:
# What are the max and min dates in the Invoice table?
cur.execute("""
SELECT max(InvoiceDate) AS max_date,
min(InvoiceDate) AS min_date
FROM Invoice""").fetchall()

[('2013-12-22 00:00:00', '2009-01-01 00:00:00')]

In [ ]:
# What tables would you need to answer "what is your most popular track?"

# Track (for track info)
# InvoiceLine (for track puchase quantity)

In [92]:
# What values from each table?

# Track.TrackId, Track.Name, Track.AlbumId
# InvoiceLine.TrackId, InvoiceLine.Quantity
# Album.AlbumId, Album.ArtistId
# Artist.ArtistId, Artist.Name

In [114]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

results = cur.execute("""
SELECT
t.name as track_name,
sum(il.quantity) AS purchase_count
FROM Track t
JOIN InvoiceLine il USING (TrackId)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""").fetchall()

print(*results, sep='\n')

('The Trooper', 5)
('Eruption', 4)
('Hallowed Be Thy Name', 4)
('Sure Know Something', 4)
('The Number Of The Beast', 4)
('Untitled', 4)
('2 Minutes To Midnight', 3)
('Blood Brothers', 3)
('Brasil', 3)
('Can I Play With Madness', 3)


In [117]:
# Advanced: get the artist who sang the song!
results = cur.execute("""
SELECT
t.name as track_name,
a.name as artist_name,
sum(il.quantity) AS purchase_count
FROM Track t
JOIN InvoiceLine il USING (TrackId)
JOIN Album USING (AlbumId)
JOIN Artist a USING (ArtistId)
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 10
""").fetchall()

print(*results, sep='\n')

('The Trooper', 'Iron Maiden', 5)
('Eruption', 'Van Halen', 4)
('Hallowed Be Thy Name', 'Iron Maiden', 4)
('Sure Know Something', 'Kiss', 4)
('The Number Of The Beast', 'Iron Maiden', 4)
('2 Minutes To Midnight', 'Iron Maiden', 3)
('Blood Brothers', 'Iron Maiden', 3)
('Can I Play With Madness', 'Iron Maiden', 3)
('Dazed and Confused', 'Led Zeppelin', 3)
('Flying High Again', 'Ozzy Osbourne', 3)


## Reflection